## File transmit through a relay server

In [1]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os

def run_transfer(b):
    source_path = source_path_input.value.strip()
    jump_host = jump_host_input.value.strip()
    jump_port = jump_port_input.value
    # 判断传输的是文件还是文件夹，选择不同的命令
    if os.path.isdir(source_path):
        cmd = f"tar cf - {source_path} | ssh -p {jump_port} root@{jump_host} 'nc localhost 3456'"
    else:
        cmd = f"cat {source_path} | ssh -p {jump_port} root@{jump_host} 'nc localhost 3456'"
    
    output_area.clear_output()
    with output_area:
        print("Executing command:")
        print(cmd)
        try:
            result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("Command finished successfully.")
            if result.stdout:
                print("Output:", result.stdout)
        except subprocess.CalledProcessError as e:
            print("Command failed with error:")
            print(e.stderr)

# 定义UI控件
source_path_input = widgets.Text(
    value='',
    placeholder='请输入文件或文件夹路径',
    description='Source:',
    layout=widgets.Layout(width='80%')
)

jump_host_input = widgets.Text(
    value='45.145.74.109',
    placeholder='跳板机IP',
    description='Jump host:',
    layout=widgets.Layout(width='50%')
)

jump_port_input = widgets.IntText(
    value=5222,
    description='SSH Port:',
    layout=widgets.Layout(width='50%')
)

transfer_button = widgets.Button(
    description='开始传输',
    button_style='success'
)

transfer_button.on_click(run_transfer)

output_area = widgets.Output()

# 显示所有控件
display(source_path_input, jump_host_input, jump_port_input, transfer_button, output_area)


Text(value='', description='Source:', layout=Layout(width='80%'), placeholder='请输入文件或文件夹路径')

Text(value='45.145.74.109', description='Jump host:', layout=Layout(width='50%'), placeholder='跳板机IP')

IntText(value=5222, description='SSH Port:', layout=Layout(width='50%'))

Button(button_style='success', description='开始传输', style=ButtonStyle())

Output()

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import subprocess
from threading import Thread
import time

# 公共参数组件
ssh_params = widgets.VBox([
    widgets.Text(description="跳板机地址:", value='45.145.74.109'),
    widgets.IntText(description="SSH端口:", value=5222),
    widgets.Text(description="用户名:", value='root'),
    widgets.IntText(description="隧道端口:", value=3456)
])

# 模式选择
mode_selector = widgets.RadioButtons(
    options=['发送端', '接收端'],
    description='模式:',
    disabled=False
)

# 文件选择组件
file_selector = widgets.FileUpload(description="选择文件", multiple=True)
folder_selector = widgets.Text(description="文件夹路径:")

# 接收端路径
receiver_path = widgets.Text(description="保存路径:", value='/path/to/received_file')

# 执行按钮
run_btn = widgets.Button(description="执行操作")
output = widgets.Output()

def on_run_clicked(b):
    with output:
        clear_output()
        params = {
            'ssh_host': ssh_params.children[0].value,
            'ssh_port': ssh_params.children[1].value,
            'user': ssh_params.children[2].value,
            'tunnel_port': ssh_params.children[3].value,
            'mode': mode_selector.value
        }
        
        if params['mode'] == '发送端':
            if len(file_selector.value) > 0:
                # 处理上传文件
                for name, content in file_selector.value.items():
                    with open(name, 'wb') as f:
                        f.write(content['content'])
                src_path = ' '.join(file_selector.value.keys())
            else:
                src_path = folder_selector.value
                
            cmd = f"tar cf - {src_path} | ssh -p {params['ssh_port']} {params['user']}@{params['ssh_host']} 'nc localhost {params['tunnel_port']}'"
            
        else:
            cmd = f"nc -l {params['tunnel_port']} | tar xf -"
            if receiver_path.value.strip():
                cmd = f"nc -l {params['tunnel_port']} > {receiver_path.value}"
            
            # 在后台启动隧道
            tunnel_cmd = f"ssh -N -R {params['tunnel_port']}:localhost:{params['tunnel_port']} -p {params['ssh_port']} {params['user']}@{params['ssh_host']}"
            Thread(target=lambda: subprocess.run(tunnel_cmd, shell=True)).start()
            time.sleep(1)  # 等待隧道建立

        print("执行命令:", cmd)
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        print(result.stdout)
        if result.stderr:
            print("错误信息:", result.stderr)

run_btn.on_click(on_run_clicked)

# 动态显示组件
def update_ui(change):
    if change['new'] == '发送端':
        special_params.children = [file_selector, folder_selector]
    else:
        special_params.children = [receiver_path]

mode_selector.observe(update_ui, names='value')

special_params = widgets.VBox()
ui = widgets.VBox([
    mode_selector,
    ssh_params,
    special_params,
    run_btn,
    output
])

# 初始UI状态
update_ui({'new': mode_selector.value})
display(ui)